# CS 640 Homework 3 - Planning with Search

In this homework, you will implement a planning algorithm based on search to play a simplified version of the [Snake](https://en.wikipedia.org/wiki/Snake_(video_game_genre)) arcade game.
In this game, you control the movement of a snake and the goal is to eat food on the board.
The snake moves by directing its head and the rest of the body follows the head.
The goal is to move the head to the food once.
Like in the arcade game, moving the snake into itself results in a failure.


## Instructions

An implementation of this game is below which provides a class to represent a current state of the game.
You should not need to change this class.

1. Implement the `plan` function further down to pick a minimum sequence of moves for the snake to get the food and achieve the goal.
2. Run the five tests at the bottom of this notebook to demonstrate your `plan` function.

Note that these tests check if your plan picks legal moves and achieves the goal.
These tests will not check if your plan used the minimum number of moves for each starting state, but that will still be part of your grade.

**Submit your updated notebook in Gradescope when you are done.**

## Baby Snake Game Implementation

In [12]:
DIRECTIONS = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}

class BabySnakeState(object):
    def __init__(self, food_pos, snake_pos):
        self.food_pos = tuple(food_pos)
        # store snake positions as tuple of (row,col), head first
        self.snake_pos = tuple(map(tuple, snake_pos))

        assert len(self.food_pos) == 2

        assert len(self.snake_pos) > 0
        assert all(len(p) == 2 for p in self.snake_pos)

    def __str__(self):
        # 5x5 textual representation. 'F' for food, 'H' for head, 'S' for body, '.' empty
        lines = [['.'] * 5 for _ in range(5)]

        # place food
        lines[self.food_pos[0]][self.food_pos[1]] = 'F'

        # place snake: head then body
        for i, (r, c) in enumerate(self.snake_pos):
            if 0 <= r < 5 and 0 <= c < 5:
                lines[r][c] = 'H' if i == 0 else 'S'
        # join rows for printing
        return '\n'.join(''.join(row) for row in lines)

    def is_goal(self):
        # goal is when the head is on the food
        return self.snake_pos[0] == self.food_pos

    def move(self, direction):
        """Return a new BabySnakeState after moving in direction (string),
           or None if the move is illegal (out of bounds or collision).
        """
        if direction not in DIRECTIONS:
            return None
        dr, dc = DIRECTIONS[direction]
        head = self.snake_pos[0]
        new_head = (head[0] + dr, head[1] + dc)

        # check bounds
        if not (0 <= new_head[0] < 5 and 0 <= new_head[1] < 5):
            return None

        # if moving into body except possibly the tail (which will move away),
        # it's a collision. We check against all parts except the last one.
        if new_head in self.snake_pos[:-1]:
            # snake collided with self
            return None

        # if new head is on food, snake grows: new head + entire previous body
        if new_head == self.food_pos:
            new_snake_pos = (new_head,) + self.snake_pos
        else:
            # normal move: new head, drop last tail
            new_snake_pos = (new_head,) + self.snake_pos[:-1]

        return BabySnakeState(self.food_pos, new_snake_pos)

    def get_successors(self):
        successors = []
        for d in sorted(DIRECTIONS.keys()):
            s2 = self.move(d)
            if s2 is not None:
                successors.append(s2)
        return successors

    def __eq__(self, other):
        return isinstance(other, BabySnakeState) and self.food_pos == other.food_pos and self.snake_pos == other.snake_pos

    def __hash__(self):
        return hash((self.food_pos, self.snake_pos))

# quick sanity test printed when cell runs
test_state = BabySnakeState([3, 3], [[1, 1], [1, 2]])
print("TEST STATE")
print(test_state)
for (i, s) in enumerate(test_state.get_successors()):
    print("SUCCESSOR", i)
    print(s)


TEST STATE
.....
.HS..
.....
...F.
.....
SUCCESSOR 0
.....
.S...
.H...
...F.
.....
SUCCESSOR 1
.....
HS...
.....
...F.
.....
SUCCESSOR 2
.....
.SH..
.....
...F.
.....
SUCCESSOR 3
.H...
.S...
.....
...F.
.....


## Random Walk through State Space

In [13]:
import random

In [14]:
random.seed(640)

s = test_state
for i in range(100):
    print("RANDOM WALK", i)
    print(s)
    if s.is_goal():
        break

    s = random.choice(s.get_successors())

RANDOM WALK 0
.....
.HS..
.....
...F.
.....
RANDOM WALK 1
.....
.S...
.H...
...F.
.....
RANDOM WALK 2
.....
.H...
.S...
...F.
.....
RANDOM WALK 3
.....
HS...
.....
...F.
.....
RANDOM WALK 4
.....
S....
H....
...F.
.....
RANDOM WALK 5
.....
.....
SH...
...F.
.....
RANDOM WALK 6
.....
.....
HS...
...F.
.....
RANDOM WALK 7
.....
H....
S....
...F.
.....
RANDOM WALK 8
.....
S....
H....
...F.
.....
RANDOM WALK 9
.....
.....
SH...
...F.
.....
RANDOM WALK 10
.....
.H...
.S...
...F.
.....
RANDOM WALK 11
.H...
.S...
.....
...F.
.....
RANDOM WALK 12
.SH..
.....
.....
...F.
.....
RANDOM WALK 13
.HS..
.....
.....
...F.
.....
RANDOM WALK 14
HS...
.....
.....
...F.
.....
RANDOM WALK 15
SH...
.....
.....
...F.
.....
RANDOM WALK 16
.S...
.H...
.....
...F.
.....
RANDOM WALK 17
.....
.SH..
.....
...F.
.....
RANDOM WALK 18
..H..
..S..
.....
...F.
.....
RANDOM WALK 19
.HS..
.....
.....
...F.
.....
RANDOM WALK 20
.S...
.H...
.....
...F.
.....
RANDOM WALK 21
.....
.S...
.H...
...F.
.....
RANDOM WALK 22
.....

## Write a Planning Function

The goal of this little snake game is to control the snake's movement so that the food is in the snake.
Modify the `plan` function below so that the snake gets the food in the minimum number of moves.

In [15]:
def plan(state):
    # YOUR CHANGES HERE

    # We'll perform a breadth-first search for a sequence of directions
    # that leads the snake's head to the food. Return a list of direction
    # strings (e.g. ['up', 'left', ...]). If no plan found, return empty list.
    from collections import deque

    if state.is_goal():
        return []

    visited = set()
    q = deque()
    q.append((state, []))
    visited.add(state)

    while q:
        cur_state, path = q.popleft()
        for d in sorted(DIRECTIONS.keys()):
            next_state = cur_state.move(d)
            if next_state is None:
                continue
            if next_state in visited:
                continue
            new_path = path + [d]
            if next_state.is_goal():
                return new_path
            visited.add(next_state)
            q.append((next_state, new_path))

    # no plan found
    return []

plan(test_state)


['down', 'down', 'right', 'right']

In [16]:
plan(test_state)

['down', 'down', 'right', 'right']

### Tests

Run the following tests to check the results of your `plan` function.
These tests will check if your plan picks legal moves and stops after getting the food.
They will not check if your plan has the minimum number of moves, but you will be graded on that.

In [17]:
def test(food_pos, snake_pos):
    state = BabySnakeState(food_pos, snake_pos)
    print("INITIAL STATE")
    print(state)
    print("")

    p = plan(state)
    for m in p:
        # make sure not moving after reaching goal
        assert state.is_goal() == False

        state = state.move(m)
        # make sure move was legal
        assert state is not None

        print("NEXT MOVE:", m)
        print(state)
        print("")

    # make sure stopped at goal
    assert state.is_goal()

    print(f"Goal achieved with {len(p)} moves.")

#### Test 1

In [18]:
test(food_pos=(0, 0), snake_pos=[(1, 0), (1, 1), (1, 2), (1, 3)])

INITIAL STATE
F....
HSSS.
.....
.....
.....

NEXT MOVE: up
H....
SSSS.
.....
.....
.....

Goal achieved with 1 moves.


#### Test 2

In [19]:
test(food_pos=(4, 2), snake_pos=[(1, 0), (1, 1), (1, 2), (1, 3)])

INITIAL STATE
.....
HSSS.
.....
.....
..F..

NEXT MOVE: down
.....
SSS..
H....
.....
..F..

NEXT MOVE: down
.....
SS...
S....
H....
..F..

NEXT MOVE: down
.....
S....
S....
S....
H.F..

NEXT MOVE: right
.....
.....
S....
S....
SHF..

NEXT MOVE: right
.....
.....
S....
S....
SSH..

Goal achieved with 5 moves.


#### Test 3

In [20]:
test(food_pos=(1, 4), snake_pos=[(1, 0), (1, 1), (1, 2), (1, 3)])

INITIAL STATE
.....
HSSSF
.....
.....
.....

NEXT MOVE: down
.....
SSS.F
H....
.....
.....

NEXT MOVE: right
.....
SS..F
SH...
.....
.....

NEXT MOVE: right
.....
S...F
SSH..
.....
.....

NEXT MOVE: right
.....
....F
SSSH.
.....
.....

NEXT MOVE: right
.....
....F
.SSSH
.....
.....

NEXT MOVE: up
.....
....H
.SSSS
.....
.....

Goal achieved with 6 moves.


#### Test 4

In [21]:
test(food_pos=(2, 2), snake_pos=[(3,4), (3, 3), (3, 2), (3, 1), (2, 1), (1, 1), (1, 2), (1, 3), (2, 3)])

INITIAL STATE
.....
.SSS.
.SFS.
.SSSH
.....

NEXT MOVE: up
.....
.SSS.
.SF.H
.SSSS
.....

NEXT MOVE: left
.....
.SS..
.SFHS
.SSSS
.....

NEXT MOVE: left
.....
.SS..
.SHSS
.SSSS
.....

Goal achieved with 3 moves.


#### Test 5

In [22]:
test(food_pos=(2,2), snake_pos=[(4, 1), (3, 1), (2, 1), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 2), (4, 2), (4, 3)])

INITIAL STATE
.....
.SSS.
.SFS.
.SSS.
.HSS.

NEXT MOVE: left
.....
.SSS.
.SFS.
.SSS.
HSS..

NEXT MOVE: up
.....
.SSS.
.SFS.
HSSS.
SS...

NEXT MOVE: up
.....
.SSS.
HSFS.
SS.S.
SS...

NEXT MOVE: up
.....
HSSS.
SSFS.
SS...
SS...

NEXT MOVE: up
H....
SSSS.
SSF..
SS...
SS...

NEXT MOVE: right
SH...
SSS..
SSF..
SS...
SS...

NEXT MOVE: right
SSH..
SS...
SSF..
SS...
SS...

NEXT MOVE: down
SSS..
S.H..
SSF..
SS...
SS...

NEXT MOVE: down
SSS..
S.S..
SSH..
SS...
SS...

Goal achieved with 9 moves.
